In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# PyTorch distributed training with Vertex AI Reduction Server

<table align="left">

  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/reduction_server/pytorch_distributed_training_reduction_server.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Run in Colab
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/reduction_server/pytorch_distributed_training_reduction_server.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/official/reduction_server/pytorch_distributed_training_reduction_server.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      Open in Vertex AI Workbench
    </a>
  </td>                                                                                               
</table>

## Overview

When you run a distributed training job across multiple nodes using GPUs, communicating gradients between nodes can contribute significant latency. Reduction Server is an all-reduce algorithm that can increase throughput and reduce latency for distributed training. This notebook demonstrates how to run a PyTorch distributed training job with Reduction Server on Vertex AI. The training job is created to fine-tune pretrained model `bert-large-cased` from the Hugging Face Transformers library on the `imdb` dataset for sentiment classification.

Learn more about [Vertex AI Training](https://cloud.google.com/vertex-ai/docs/training/custom-training) and [Vertex AI Reduction Server](https://cloud.google.com/blog/topics/developers-practitioners/optimize-training-performance-reduction-server-vertex-ai).

### Objective

In this notebook, you learn how to create a PyTorch distributed training job that uses PyTorch distributed training framework and tools, and run the training job on the Vertex AI Training service with Reduction Server.

This tutorial uses the following Google Cloud ML services and resources:

* Vertex AI Training
* Cloud Storage


The steps performed include:

* Create a PyTorch distributed training application
* Package the training application with pre-built containers
* Create a custom job on Vertex AI with Reduction Server
* Submit and monitor the job 


### Dataset

In this tutorial, we use [`imdb`](https://huggingface.co/datasets/imdb) dataset from Hugging Face. `imdb` is a large movie review dataset for binary sentiment classification containing a set of 25,000 highly polar movie reviews for training, and 25,000 for testing.

### Costs

This tutorial uses billable components of Google Cloud:

* Vertex AI
* Cloud Storage

Learn about [Vertex AI
pricing](https://cloud.google.com/vertex-ai/pricing) and [Cloud Storage
pricing](https://cloud.google.com/storage/pricing), and use the [Pricing
Calculator](https://cloud.google.com/products/calculator/)
to generate a cost estimate based on your projected usage.

### Set up your local development environment

**If you are using Colab or Vertex AI Workbench Notebooks**, your environment already meets
all the requirements to run this notebook.

**Otherwise**, make sure your environment meets this notebook's requirements.
You need the following:

* The Google Cloud SDK

The Google Cloud guide to [Setting up a Python development
environment](https://cloud.google.com/python/setup) and the [Jupyter
installation guide](https://jupyter.org/install) provide detailed instructions
for meeting these requirements. The following steps provide a condensed set of
instructions:

1. [Install and initialize the Cloud SDK.](https://cloud.google.com/sdk/docs/)

1. [Install Python 3.](https://cloud.google.com/python/setup#installing_python)

1. [Install
   virtualenv](https://cloud.google.com/python/setup#installing_and_using_virtualenv)
   and create a virtual environment that uses Python 3. Activate the virtual environment.

1. To install Jupyter, run `pip3 install jupyter` on the
command-line in a terminal shell.

1. To launch Jupyter, run `jupyter notebook` on the command-line in a terminal shell.

1. Open this notebook in the Jupyter Notebook Dashboard.

## Installation

Install the following packages required to execute this notebook.

In [ ]:
import os

# The Vertex AI Workbench Notebook product has specific requirements
IS_WORKBENCH_NOTEBOOK = os.getenv("DL_ANACONDA_HOME")
IS_USER_MANAGED_WORKBENCH_NOTEBOOK = os.path.exists(
    "/opt/deeplearning/metadata/env_version"
)

# Vertex AI Notebook requires dependencies to be installed with '--user'
USER_FLAG = ""
if IS_WORKBENCH_NOTEBOOK:
    USER_FLAG = "--user"

! pip3 install --upgrade google-cloud-aiplatform {USER_FLAG} -q

### Restart the kernel

After you install the additional packages, you need to restart the notebook kernel so it can find the packages.

In [ ]:
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

## Before you begin

### Set up your Google Cloud project

**The following steps are required, regardless of your notebook environment.**

1. [Select or create a Google Cloud project](https://console.cloud.google.com/cloud-resource-manager). When you first create an account, you get a $300 free credit towards your compute/storage costs.

1. [Make sure that billing is enabled for your project](https://cloud.google.com/billing/docs/how-to/modify-project).

1. [Enable the Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com).

1. If you are running this notebook locally, you need to install the [Cloud SDK](https://cloud.google.com/sdk).

1. Enter your project ID in the cell below. Then run the cell to make sure the
Cloud SDK uses the right project for all the commands in this notebook.

**Note**: Jupyter runs lines prefixed with `!` as shell commands, and it interpolates Python variables prefixed with `$` into these commands.

#### Set your project ID

**If you don't know your project ID**, you may be able to get your project ID using `gcloud`.

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

In [ ]:
if PROJECT_ID == "" or PROJECT_ID is None or PROJECT_ID == "[your-project-id]":
    # Get your GCP project id from gcloud
    shell_output = ! gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID:", PROJECT_ID)

In [ ]:
! gcloud config set project $PROJECT_ID

#### Region

You can also change the `REGION` variable, which is used for operations
throughout the rest of this notebook.  Below are regions supported for Vertex AI. It is recommended that you choose the region closest to you.

- Americas: `us-central1`
- Europe: `europe-west4`
- Asia Pacific: `asia-east1`

You may not use a multi-regional bucket for training with Vertex AI. Not all regions provide support for all Vertex AI services.

Learn more about [Vertex AI regions](https://cloud.google.com/vertex-ai/docs/general/locations).

In [ ]:
REGION = "[your-region]"  # @param {type: "string"}

if REGION == "[your-region]":
    REGION = "us-central1"

#### UUID

If you are in a live tutorial session, you might be using a shared test account or project. To avoid name collisions between users on resources created, you create a uuid for each instance session, and append it onto the name of resources you create in this tutorial.

In [ ]:
import random
import string


# Generate a uuid of a specified length(default=8)
def generate_uuid(length: int = 8) -> str:
    return "".join(random.choices(string.ascii_lowercase + string.digits, k=length))


UUID = generate_uuid()

### Authenticate your Google Cloud account

**If you are using Vertex AI Workbench Notebooks**, your environment is already
authenticated. 

**If you are using Colab**, run the cell below and follow the instructions
when prompted to authenticate your account via oAuth.

**Otherwise**, follow these steps:

1. In the Cloud Console, go to the [**Create service account key**
   page](https://console.cloud.google.com/apis/credentials/serviceaccountkey).

2. Click **Create service account**.

3. In the **Service account name** field, enter a name, and
   click **Create**.

4. In the **Grant this service account access to project** section, click the **Role** drop-down list. Type "Vertex AI"
into the filter box, and select
   **Vertex AI Administrator**. Type "Storage Object Admin" into the filter box, and select **Storage Object Admin**.

5. Click *Create*. A JSON file that contains your key downloads to your
local environment.

6. Enter the path to your service account key as the
`GOOGLE_APPLICATION_CREDENTIALS` variable in the cell below and run the cell.

In [ ]:
# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

import os
import sys

# If on Vertex AI Workbench, then don't execute this code
IS_COLAB = "google.colab" in sys.modules
if not os.path.exists("/opt/deeplearning/metadata/env_version") and not os.getenv(
    "DL_ANACONDA_HOME"
):
    if "google.colab" in sys.modules:
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this notebook locally, replace the string below with the
    # path to your service account key and run this cell to authenticate your GCP
    # account.
    elif not os.getenv("IS_TESTING"):
        %env GOOGLE_APPLICATION_CREDENTIALS '[your-service-account-key-path]'

### Create a Cloud Storage bucket

**The following steps are required, regardless of your notebook environment.**

When you submit a training job using the Cloud SDK, you upload a Python package
containing your training code to a Cloud Storage bucket. Vertex AI runs
the code from this package. In this tutorial, Vertex AI also saves the
trained model that results from your job in the same bucket. Using this model artifact, you can then create Vertex AI model resources.

Set the name of your Cloud Storage bucket below. It must be unique across all
Cloud Storage buckets.

You may also change the `REGION` variable, which is used for operations
throughout the rest of this notebook. Make sure to [choose a region where Vertex AI services are
available](https://cloud.google.com/vertex-ai/docs/general/locations#available_regions). You may
not use a Multi-Regional Storage bucket for training with Vertex AI.

In [ ]:
BUCKET_NAME = "[your-bucket-name]"  # @param {type:"string"}
BUCKET_URI = f"gs://{BUCKET_NAME}"

In [ ]:
if BUCKET_NAME == "" or BUCKET_NAME is None or BUCKET_NAME == "[your-bucket-name]":
    BUCKET_NAME = PROJECT_ID + "aip-" + UUID
    BUCKET_URI = f"gs://{BUCKET_NAME}"

**Only if your bucket doesn't already exist**: Run the following cell to create your Cloud Storage bucket.

In [ ]:
! gsutil mb -l $REGION -p $PROJECT_ID $BUCKET_URI

Validate access to your Cloud Storage bucket by examining its contents

In [ ]:
! gsutil ls -al $BUCKET_URI

Finally, print out the variables used throughout the tutorial.

In [ ]:
print(f"PROJECT_ID = {PROJECT_ID}")
print(f"BUCKET_URI = {BUCKET_URI}")
print(f"REGION = {REGION}")
print(f"UUID = {UUID}")

### Import libraries

In [ ]:
from google.cloud import aiplatform

### Initialize Vertex AI SDK for Python

Initialize the Vertex AI SDK for Python for your project and corresponding bucket.

In [ ]:
aiplatform.init(project=PROJECT_ID, location=REGION, staging_bucket=BUCKET_URI)

# Tutorial

Now you are ready to start creating the PyTorch distributed training job.

### Packaging the training application

Before running the training job on Vertex AI, the training application code and any dependencies must be packaged and uploaded to Cloud Storage bucket or Container Registry or Artifact Registry that your Google Cloud project can access. This section shows how to package and stage your application in the cloud.

There are two ways to package your application and dependencies and train on Vertex AI:

1. [Create a Python source distribution](https://cloud.google.com/vertex-ai/docs/training/create-python-pre-built-container) with the training code and dependencies to use with a [pre-built container](https://cloud.google.com/vertex-ai/docs/training/pre-built-containers) on Vertex AI
2. Use [custom containers](https://cloud.google.com/ai-platform/training/docs/custom-containers-training) to package dependencies using Docker containers

**This notebook shows the Python source distribution option to run a custom training job on Vertex AI.**

#### Recommended training application structure

You can structure your training application in any way you like. However, the [following structure](https://cloud.google.com/vertex-ai/docs/training/create-python-pre-built-container#structure) is commonly used in Vertex AI samples, and having your project's organization be similar to the samples can make it easier for you to follow the samples.

```
.
├── python_package
│   ├── README.md
│   ├── setup.py
│   └── trainer
│       ├── __init__.py
│       └── task.py
└── pytorch-distributed-training-reduction-server.ipynb    --> This notebook
```

1. Main project directory contains your `setup.py` file with the dependencies. 
2. Use a subdirectory named `trainer` to store your main application module and `scripts` to submit training jobs locally or cloud
3. Inside `trainer` directory:
    - `task.py` - Main application module 1) initializes PyTorch distributed training environment, and 2) Runs the model training and evaluation experiment, and exports the final model.
    - `__init__.py` is required to make Python treat directories containing the file as packages.

#### Define variables for the training application

Initialize the variables to define pre-built container image, location of training application and training module.

In [ ]:
APP_NAME = "pytorch-bert"

PRE_BUILT_TRAINING_CONTAINER_IMAGE_URI = (
    "us-docker.pkg.dev/vertex-ai/training/pytorch-gpu.1-9:latest"
)

PYTHON_PACKAGE_APPLICATION_DIR = "python_package"

source_package_file_name = f"{PYTHON_PACKAGE_APPLICATION_DIR}/dist/trainer-0.1.tar.gz"
python_package_gcs_uri = (
    f"{BUCKET_URI}/pytorch-on-gcp/{APP_NAME}/train/python_package/trainer-0.1.tar.gz"
)

python_module_name = "trainer.task"

#### Create file structure of the training application

In [ ]:
! mkdir {PYTHON_PACKAGE_APPLICATION_DIR}
! touch {PYTHON_PACKAGE_APPLICATION_DIR}/README.md

! mkdir {PYTHON_PACKAGE_APPLICATION_DIR}/trainer
! touch {PYTHON_PACKAGE_APPLICATION_DIR}/trainer/__init__.py

#### Create the `setup.py` file for the training application

Following is the `setup.py` file for the training application. The `find_packages()` function inside `setup.py` includes the `trainer` directory in the package as it contains `__init__.py` which tells [Python Setuptools](https://setuptools.readthedocs.io/en/latest/) to include all subdirectories of the parent directory as dependencies. 

In [ ]:
%%writefile ./{PYTHON_PACKAGE_APPLICATION_DIR}/setup.py

import os
from setuptools import find_packages
from setuptools import setup
import setuptools

from distutils.command.build import build as _build
import subprocess


REQUIRED_PACKAGES = [
    'transformers',
    'datasets',
    'evaluate',
]

setup(
    name='trainer',
    version='0.1',
    install_requires=REQUIRED_PACKAGES,
    packages=find_packages(),
    include_package_data=True,
    description='Vertex AI | Training | PyTorch | Text Classification | Python Package'
)


#### Create training application code

`task.py` is the main application module. It initializes the PyTorch distributed training environment and runs the model training and evaluation experiment, and exports the final model.


In [ ]:
%%writefile ./{PYTHON_PACKAGE_APPLICATION_DIR}/trainer/task.py
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the \"License\");
# you may not use this file except in compliance with the License.\n",
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an \"AS IS\" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

import os
import numpy as np
import pandas as pd
from datetime import datetime
import argparse

import torch
import torch.distributed as dist
torch.cuda.empty_cache()

import datasets
from datasets import ClassLabel, Sequence, load_dataset

import transformers
from transformers import (
    AutoModelForSequenceClassification, 
    AutoTokenizer,
    EvalPrediction, 
    Trainer, 
    TrainingArguments,
    default_data_collator)

from google.cloud import storage


def main():

  parser = argparse.ArgumentParser(formatter_class=argparse.ArgumentDefaultsHelpFormatter)
  parser.add_argument("--epochs", type=int, help="Number of training epochs.", default=2)
  parser.add_argument("--batch_size", type=int, help="Training batch size for one process.", default=32)
  parser.add_argument("--model_dir", type=str, help="Directory for saving models.", default=os.environ['AIP_MODEL_DIR'] if 'AIP_MODEL_DIR' in os.environ else "")
  argv = parser.parse_args()

  model_name_or_path = "bert-large-uncased"
  padding = "max_length"
  max_seq_length = 128

  datasets = load_dataset("imdb")
  label_list = datasets["train"].unique("label")
  label_to_id = {1: 1, 0: 0, -1: 0}

  tokenizer = AutoTokenizer.from_pretrained(
      model_name_or_path,
      use_fast=True,
  )

  def preprocess_function(examples):
      """
      Tokenize the input example texts
      """
      args = (examples["text"],)
      result = tokenizer(
          *args, padding=padding, max_length=max_seq_length, truncation=True
      )

      # Map labels to IDs (not necessary for GLUE tasks)
      if label_to_id is not None and "label" in examples:
          result["label"] = [label_to_id[example] for example in examples["label"]]

      return result

  # apply preprocessing function to input examples
  datasets = datasets.map(preprocess_function, batched=True, load_from_cache_file=True)

  model = AutoModelForSequenceClassification.from_pretrained(
      model_name_or_path, 
      num_labels=len(label_list)
  )

  ngpus_per_node = torch.cuda.device_count()
  world_size = int(os.environ["WORLD_SIZE"])
  
  # Since we have ngpus_per_node processes per node, the total world_size
  # needs to be adjusted accordingly
  world_size =  world_size * ngpus_per_node

  start = datetime.now().strftime("%Y_%m_%d_%H_%M_%S")
  print(f'Starting distributed training: {start}') 
  
  # Use torch.multiprocessing.spawn to launch distributed processes
  torch.multiprocessing.spawn(main_worker,
    args = (ngpus_per_node, world_size, datasets, model, tokenizer, argv),
    nprocs = ngpus_per_node,
    join = True)
  
  end = datetime.now().strftime("%Y_%m_%d_%H_%M_%S")
  print(f'Distributed training complete: {end}')

def main_worker(local_rank, ngpus_per_node, world_size, datasets, model, tokenizer, argv):

  # This is the (global) rank of the current process
  rank = int(os.environ["RANK"])
  
  # For multiprocessing distributed training, rank needs to be the
  # global rank among all the processes
  rank = rank * ngpus_per_node + local_rank
  print (f"Distributed and Multi-processing. Setting rank for each worker. rank={rank}")

  dist.init_process_group(
      backend="nccl", 
      init_method="env://",
      world_size=world_size, 
      rank=rank)
  
  per_device_batch_size = int(argv.batch_size / ngpus_per_node)

  training_args = TrainingArguments(
      output_dir="/tmp/output/",
      num_train_epochs=argv.epochs, 
      per_device_train_batch_size=per_device_batch_size,
      per_device_eval_batch_size=per_device_batch_size,
      local_rank=local_rank,
  )

  def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions, tuple) else p.predictions
    preds = np.argmax(preds, axis=1)
    return {"accuracy": (preds == p.label_ids).astype(np.float32).mean().item()}
  
  trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=datasets["train"],
    eval_dataset=datasets["test"],
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
    data_collator=default_data_collator,
  )

  trainer.train()

  # Save the trained model locally
  model_filename = "pytorch-birt-model"
  local_path = os.path.join("/tmp", model_filename)
  trainer.save_model(local_path)

  if (os.path.exists(local_path)):
    # Upload the trained model to Cloud storage
    model_directory = argv.model_dir
    storage_path = os.path.join(model_directory, model_filename)
    blob = storage.blob.Blob.from_string(storage_path, client=storage.Client())

    files = [f for f in os.listdir(local_path) if os.path.isfile(os.path.join(local_path, f))]
    for file in files:
      local_file = os.path.join(local_path, file)
      blob.upload_from_filename(local_file)
  
    print(f"Saved model files in {model_directory}/{model_filename}")


if __name__ == "__main__":
    main()


#### Create a source distribution

Create a source distribution `dist/trainer-0.1.tar.gz`, and upload the source distribution with training application to Cloud Storage bucket, and then validate the source distribution exists on Cloud Storage bucket.

In [ ]:
! cd {PYTHON_PACKAGE_APPLICATION_DIR} && python3 setup.py sdist --formats=gztar

! gsutil cp {source_package_file_name} {python_package_gcs_uri}

! gsutil ls -l {python_package_gcs_uri}

### Run custom training job with Reduction Server on Vertex AI

Configure a custom job with the pre-built container image for PyTorch and training code packaged as Python source distribution.

In [ ]:
print(f"APP_NAME={APP_NAME}")
print(
    f"PRE_BUILT_TRAINING_CONTAINER_IMAGE_URI={PRE_BUILT_TRAINING_CONTAINER_IMAGE_URI}"
)
print(f"python_package_gcs_uri={python_package_gcs_uri}")
print(f"python_module_name={python_module_name}")

#### Create a training job

You use [Vertex AI SDK for Python](https://cloud.google.com/vertex-ai/docs/start/client-libraries#client_libraries) to create a custom training job.

In [ ]:
JOB_NAME = f"pytorch-birt-reduction-server-{UUID}"
print(f"JOB_NAME={JOB_NAME}")

job = aiplatform.CustomPythonPackageTrainingJob(
    display_name=f"{JOB_NAME}",
    python_package_gcs_uri=python_package_gcs_uri,
    python_module_name=python_module_name,
    container_uri=PRE_BUILT_TRAINING_CONTAINER_IMAGE_URI,
)

#### Define the training cluster worker pool and experiment configuration parameters

Reduction Server can be used with any distributed training framework that uses the NVIDIA NCCL library for the all-reduce collective operation. You do not need to change or recompile your training application.

The Google Cloud guide [Distributed training](https://cloud.google.com/vertex-ai/docs/training/distributed-training) provides detailed instructions on how to run distributed training jobs on Vertex AI.

In [ ]:
# Training cluster worker pool configuration
REPLICA_COUNT = 3
MACHINE_TYPE = "n1-standard-16"
ACCELERATOR_TYPE = "NVIDIA_TESLA_V100"
ACCELERATOR_COUNT = 2

# Reduction Server configuration
REDUCTION_SERVER_COUNT = 4
REDUCTION_SERVER_MACHINE_TYPE = "n1-highcpu-16"
REDUCTION_SERVER_IMAGE_URI = (
    "us-docker.pkg.dev/vertex-ai-restricted/training/reductionserver:latest"
)
ENVIRONMENT_VARIABLES = {"NCCL_DEBUG": "INFO"}

# Training experiment parameters
EPOCHS = 2
BATCH_SIZE = 32
MODEL_DIR = f"{BUCKET_URI}/{JOB_NAME}"

training_args = [
    "--epochs",
    str(EPOCHS),
    "--batch_size",
    str(BATCH_SIZE),
    "--model_dir",
    MODEL_DIR,
]

#### Submit the training job

After the training cluster configuration parameters have been defined, use the Vertex AI SDK for Python to submit and monitor a training job.

*NOTE: When using Vertex AI SDK for Python for submitting a training job, it creates a Training Pipeline which launches the custom job on Vertex AI Training service.*

In [ ]:
model = job.run(
    replica_count=REPLICA_COUNT,
    machine_type=MACHINE_TYPE,
    accelerator_type=ACCELERATOR_TYPE,
    accelerator_count=ACCELERATOR_COUNT,
    reduction_server_replica_count=REDUCTION_SERVER_COUNT,
    reduction_server_machine_type=REDUCTION_SERVER_MACHINE_TYPE,
    reduction_server_container_uri=REDUCTION_SERVER_IMAGE_URI,
    environment_variables=ENVIRONMENT_VARIABLES,
    args=training_args,
    sync=True,
)

#### Monitor the training job (optional)

You can monitor the custom job launched from Cloud Console following the link [here](https://console.cloud.google.com/vertex-ai/training/training-pipelines/) or use gcloud CLI command [`gcloud beta ai custom-jobs stream-logs`](https://cloud.google.com/sdk/gcloud/reference/beta/ai/custom-jobs/stream-logs)

#### Validate the model artifacts (optional)

You can validate the model artifacts written to Cloud Storage bucket by the training application after the job completes successfully.

In [ ]:
print(f"Model artifacts are available at {MODEL_DIR}")

# Cleaning up

To clean up all Google Cloud resources used in this project, you can [delete the Google Cloud project](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects) you used for the tutorial.

Otherwise, you can delete the individual resources you created in this tutorial.

In [ ]:
delete_custom_job = True
delete_bucket = False

if delete_custom_job:
    try:
        job.delete()
    except Exception as e:
        print(e)

if delete_bucket and "BUCKET_URI" in globals():
    ! gsutil rm -r $BUCKET_URI